In [2]:
import os
import rasterio
import numpy as np
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import shape
import xml.etree.ElementTree as ET
from rasterio.features import shapes



def buffer_into_polygon(gdf, buffer_size):

    gdf['geometry'] = gdf.geometry.buffer(buffer_size)
    gdf_cleaned = gdf[~gdf['geometry'].is_empty & gdf['geometry'].is_valid]

    # Keep only the polygons with the biggest area
    #gdf_cleaned['area'] = gdf_cleaned['geometry'].area
    #gdf_largest = gdf_cleaned.sort_values(by='area', ascending=False).head(1)

    return gdf_cleaned
    
def create_grid(size, pixel_size):
    """
    Create a grid with a given size and pixel resolution.
    """
    center = size // 2
    x, y = np.meshgrid(np.arange(size), np.arange(size))
    distance_map = np.maximum(np.abs(x - center), np.abs(y - center)) * pixel_size
    return distance_map

def xml_to_dict(element):

    if len(element) == 0:
        return element.text
    
    result = {}
    
    for child in element:

        child_dict = xml_to_dict(child)

        if child.tag in result:
            if isinstance(result[child.tag], list):
                result[child.tag].append(child_dict)
            else:
                result[child.tag] = [result[child.tag], child_dict]
        else:
            result[child.tag] = child_dict
            
    return result

In [3]:
def atmospheric_point_scattering_function(atmosphere_parameters, band):
    
    """
    Calculates the Fr weight per distance.
    """
    
    # Converts the grid distance to km:
    if atmosphere_parameters["General_Info"]["satellite"] == "MSI_S2":
        if band == "B11": # 20m
            grid_matrix = create_grid(30, 20)
        else: # 10m
            grid_matrix = create_grid(60, 10)
    else: # 30m   
        grid_matrix = create_grid(20, 30)
    
    radius_km = grid_matrix / 1000
    
    # Zenith view angle -> degree:
    view_z = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['view_z'])
    
    # Rayleigh UPWARD diffuse transmittance -> T_upward_difRayleigh:
    Rayleigh_OpticalDepth = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['optical_depth__total_Ray'])
    T_upward_Rayleigh = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['rayleigh_scatransmi_upward'])
    T_upward_dirRayleigh = np.exp(-Rayleigh_OpticalDepth / np.cos(view_z * (np.pi / 180)))
    T_upward_difRayleigh = T_upward_Rayleigh - T_upward_dirRayleigh
    
    # Aerosol UPWARD diffuse transmittance -> T_upward_difAerosol:
    Aerosol_OpticalDepth = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['optical_depth__total_Aero'])
    T_upward_Aerosol = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['aerosol_scatransmi_upward'])
    T_upward_dirAerosol = np.exp(-Aerosol_OpticalDepth / np.cos(view_z * (np.pi / 180)))
    T_upward_difAerosol = T_upward_Aerosol - T_upward_dirAerosol
    
    # Calculates the Aerosol's Fr and Rayleigh's Fr functions using the equation described by Vermote et al.(2006):
    FrRayleigh = ((0.930 * np.exp(-0.08 * radius_km)) + (0.070 * np.exp(-1.10 * radius_km)))
    FrAerosol = ((0.448 * np.exp(-0.27 * radius_km)) + (0.552 * np.exp(-2.83 * radius_km)))
    
    # Calculates the APSF (Fr) -> Atmospheric Point Scattering Function:
    Fr = (T_upward_difRayleigh * FrRayleigh + T_upward_difAerosol * FrAerosol) / (T_upward_difRayleigh + T_upward_difAerosol)
    
    return Fr

def Adjacency_correction(atmosphere_parameters, band, array_band, adjc_array):
    
    """
    Removes the adjacency effect of the image, using the equation described in the Vermote et al. (1997).
    """
        
    # Zenith view angle -> degree:
    view_z = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['view_z'])
    
    # Atmopheric optical depth (Rayleigh + Aerosol) -> Atmospheric_OpticalDepth:
    Atmospheric_OpticalDepth = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['optical_depth__total_AeroRay'])
    
    # Total transmittance UPWARD (Rayleigh + Aerosol) -> T_upward:
    T_upward = float(atmosphere_parameters["InputData"]['sixSV_params'][band]['total_scattering_transmittance_upward'])
    
    # Total transmittance UPWARD direct (Rayleigh + Aerosol) -> T_upward_dirAeroRay:
    T_upward_dirAeroRay = np.exp(-Atmospheric_OpticalDepth / np.cos(view_z * (np.pi / 180)))
    
    # Total transmittance UPWARD diffuse (Rayleigh + Aerosol) -> T_upward_difAeroRay
    T_upward_difAeroRay = T_upward - T_upward_dirAeroRay
    
    # Surface reflectance without adjacency effect - Vermote et al. (1997):
    sr_corr = array_band - (adjc_array * T_upward_difAeroRay)
    
    return sr_corr


def raster2shp(raster_numpy, raster_transform, raster_crs):

    results = list(
            {"properties": {"raster_val": v}, "geometry": s}
            for s, v in shapes(np.asarray(raster_numpy, dtype=np.int16), transform=raster_transform)
            if v  # Only take shapes with raster_val = True (i.e., v=1)
    )
    geometries = [shape(feature["geometry"]) for feature in results]
    
    gdf = gpd.GeoDataFrame(geometry=geometries, crs=raster_crs)
    
    gdf_copy = gdf.copy().to_crs(3857)
    
    gdf_copy['area'] = gdf_copy['geometry'].area
    
    gdf_largest = gdf_copy.nlargest(5, 'area')
    
    gdf_final = gdf_largest.to_crs(raster_crs)
    
    return gdf_final

def get_mask_water(img_path):

    band_path = [i for i in os.listdir(img_path)]
    green_band = next((band for band in band_path if "B3" in band or "B03" in band), None)
    swir_band = next((band for band in band_path if "B11" in band or "B6" in band or "B06" in band), None)

    xda_green = rxr.open_rasterio(os.path.join(img_path, green_band))
    xda_swir = rxr.open_rasterio(os.path.join(img_path, swir_band))

    xda_green_matched = xda_green.rio.reproject_match(xda_swir)

    mndwi = (xda_green_matched - xda_swir) / (xda_green_matched + xda_swir)
    mndwi_mask = mndwi > 0.2
    water_mask = mndwi_mask

    with rasterio.open(os.path.join(img_path, green_band)) as src:
        _transform = src.transform
        _crs = src.crs

    # Include adjacency buffer
    water_shp = raster2shp(water_mask, _transform, _crs)
    water_copy = water_shp.copy().to_crs(3857)
    
    water_shp_buffered = buffer_into_polygon(water_copy, -3000).to_csv(water_shp.crs)
    
    # Get only the difference between the original and the buffered
    water_shp_final = gpd.GeoDataFrame(geometry=water_shp.difference(water_shp_buffered), crs=water_shp.crs)
    
    #water_shp_final_copy = water_shp_final.copy()
    #water_adjc = buffer_into_polygon(water_shp_final_copy, 500)

    return water_shp_final

In [5]:
import os
import numpy as np
import rioxarray as rxr
from rasterio.mask import mask
from scipy.signal import convolve2d
from scipy.ndimage import binary_dilation as bn

image_path = r"Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\atmcor"

output_path = r"Z:\guser\tml\mypapers\HLS_package_paper\adjacent_correction\21HVB_20230209\v2"

tree = ET.parse(r'Z:\guser\tml\mypapers\HLS_package_paper\package_validation\sentinel\21HVB\atmcor\MTD.xml')

root = tree.getroot()
metadata_6sv = xml_to_dict(root)

def create_inward_buffer(input_raster_path, buffer_distance_meters = 3000, pixel_resolution=30):
    
    band_path = [i for i in os.listdir(input_raster_path)]
    green_band = next((band for band in band_path if "B3" in band or "B03" in band), None)
    swir_band = next((band for band in band_path if "B11" in band or "B6" in band or "B06" in band), None)

    xda_green = rxr.open_rasterio(os.path.join(input_raster_path, green_band))
    xda_swir = rxr.open_rasterio(os.path.join(input_raster_path, swir_band))

    xda_green_matched = xda_green.rio.reproject_match(xda_swir)

    mndwi = (xda_green_matched - xda_swir) / (xda_green_matched + xda_swir)
    mndwi_mask = mndwi > 0.2
    water_mask = mndwi_mask.astype(int).values[0,:,:]
    
    # Calculate buffer in pixels (buffer distance converted to pixels)
    buffer_pixels = int(abs(buffer_distance_meters) / pixel_resolution)

    # Invert the water mask (0 -> water, 1 -> non-water)
    inverted_mask = np.where(water_mask == 1, 0, 1)

    # Apply dilation to the inverted mask to expand the non-water area (expand the outer non-water body)
    dilated_mask = bn(inverted_mask, structure=np.ones((buffer_pixels, buffer_pixels)))

    # Invert back the dilated mask to create the inward buffer (1 for inside the buffer, 0 for outside)
    buffer_mask = np.where(np.logical_and(dilated_mask == 1, water_mask == 1), 1, 0)

    return buffer_mask

# Identify the pixels in which we're gonna apply the adjacent correction
# The first gdf is the water mask, the second is the buffer. We will apply the correction in the buffer, and then clip the water mask to drop pixels in the border.
#water_shp_final = get_mask_water(image_path)

polygon_mask_binary = create_inward_buffer(image_path)

# Selecting only those important band for AQUAVis
if metadata_6sv["General_Info"]["satellite"] == "MSI_S2":
    
    aquavis_bands = ["B2", "B02", "B3", "B03", "B4", "B04", "B5", "B05", "B8A", "B8A", "B11"]
    filtered_dict = {key: value for key, value in metadata_6sv["General_Info"]["bandname"].items() if any(band in value for band in aquavis_bands)}
    
    updated_dict = {key: value.replace(".jp2", ".TIF") for key, value in filtered_dict.items()}
    filtered_dict = updated_dict
    band_names = list(filtered_dict.values())
    
    pixel_resolution = [10, 20]
    
else:
    
    aquavis_bands = ["B2", "B02", "B3", "B03", "B4", "B04", "B5", "B05", "B6", "B06"]
    filtered_dict = {key: value for key, value in metadata_6sv["General_Info"]["bandname"].items() if any(band in value for band in aquavis_bands)}
    
    band_names = list(filtered_dict.values())
    
    pixel_resolution = 30
    
band_index = list(filtered_dict.keys())

for i in range(len(band_index)):
    
    print(f"Processing band {band_names[i]}")
    
    band_key = band_index[i]
    
    # Environmental function
    Fr = atmospheric_point_scattering_function(metadata_6sv, band_key)
    
    # Surface Reflectance image without the adjacent correction
    with rasterio.open(os.path.join(image_path, filtered_dict[band_key])) as src:
        image_rs_no_adjc = src.read(1)
    
        image_rs_no_adjc = np.where(image_rs_no_adjc == -9999, np.nan, image_rs_no_adjc)
        image_rs_no_adjc = np.where(image_rs_no_adjc == 0, np.nan, image_rs_no_adjc)
        
        crs_image = src.crs
        transform = src.transform 
        nodata_value = src.nodata
        
        # if len(water_shp_final) != 0: # in case of water_shp_final return empty
        #      polygon_mask, _ = mask(src, water_shp_final.geometry, crop=False, nodata=src.nodata)
        #      polygon_mask = polygon_mask[0,:,:]
        #      polygon_mask_binary = np.where(polygon_mask == 0, 0, 1)
        #  
        # else:
        #      polygon_mask_binary = np.zeros_like(image_rs_no_adjc)
            
    # Apply the convolution to calculate the <p>
    convolved_image = convolve2d(image_rs_no_adjc, Fr, mode='same', boundary='fill', fillvalue=0)
    
    one_value_array = np.full_like(image_rs_no_adjc, 1)
    convolved_Fr = convolve2d(one_value_array, Fr, mode='same', boundary='fill', fillvalue=0)
    
    rho_env = convolved_image / convolved_Fr
    
    # Apply the adjacency correction
    sr_corr = Adjacency_correction(metadata_6sv, band_key, image_rs_no_adjc, rho_env)
    
    # Only update pixels where mask == 1
    adjc_sr = np.where(polygon_mask_binary == 1, sr_corr, image_rs_no_adjc)
    
    output_tif = os.path.join(output_path, f"2_corrected_band_{band_names[i]}")
    
    # Open a new file to write the adjusted raster
    with rasterio.open(output_tif, 'w', driver='GTiff', 
                       count=1, dtype=adjc_sr.dtype, 
                       width=adjc_sr.shape[1], height=adjc_sr.shape[0],
                       crs=crs_image, transform=transform, nodata=nodata_value) as dst:
        dst.write(adjc_sr, 1)  # Write the data to the file
        

Processing band T21HVB_20230209T133831_B02.TIF


ValueError: operands could not be broadcast together with shapes (5490,5490) (10980,10980) (10980,10980) 

In [11]:
import numpy as np
import rasterio
from scipy.ndimage import binary_dilation

def create_inward_buffer(input_raster_path, buffer_distance_meters, output_raster_path, pixel_resolution=30):
    
    # Calculate buffer in pixels (buffer distance converted to pixels)
    buffer_pixels = int(abs(buffer_distance_meters) / pixel_resolution)

    # Read the input raster (binary mask)
    with rasterio.open(input_raster_path) as src:
        water_mask = src.read(1)  # Read the first band (assuming it's a single-band binary mask)
        profile = src.profile

    # Invert the water mask (0 -> water, 1 -> non-water)
    inverted_mask = np.where(water_mask == 1, 0, 1)

    # Apply dilation to the inverted mask to expand the non-water area (expand the outer non-water body)
    dilated_mask = binary_dilation(inverted_mask, structure=np.ones((buffer_pixels, buffer_pixels)))

    # Invert back the dilated mask to create the inward buffer (1 for inside the buffer, 0 for outside)
    buffer_mask = np.where(np.logical_and(dilated_mask == 1, water_mask == 1), 1, 0)

    # Now, the buffer_mask will have 1s for the pixels inside the buffer zone, and 0s for outside.

    # Update the profile for output (same as input, but changing the data type)
    profile.update(dtype=rasterio.uint8, count=1, nodata=0)

    # Write the result to an output raster
    with rasterio.open(output_raster_path, 'w', **profile) as dst:
        dst.write(buffer_mask, 1)

    print(f"Buffer raster saved at: {output_raster_path}")

# Example usage
input_raster_path = r'C:\Users\tml411\Documents\Python Scripts\binary.tiff'  # Path to the binary water body mask
buffer_distance_meters = -3000  # Inward buffer distance in meters
output_raster_path = r'C:\Users\tml411\Documents\Python Scripts\binary_buffer.tiff'   # Output path for the buffer raster

create_inward_buffer(input_raster_path, buffer_distance_meters, output_raster_path)


Buffer raster saved at: C:\Users\tml411\Documents\Python Scripts\binary_buffer.tiff
